In [1]:
from __future__ import division

import pylab as plt
import numpy as np
import tensorflow as tf
import cv2
import time
import os

import sys


# modulabs setting
sys.path.insert(0, "/home/modu/DemoDay_ImageProcessing_Team/gwansang/GenderRecognition")
sys.path.insert(0, "/home/modu/DemoDay_ImageProcessing_Team/gwansang/AgeEstimation/")
sys.path.insert(0, "/home/modu/DemoDay_ImageProcessing_Team/gwansang/EmotionEstimation/")

# my computer setting
#sys.path.insert(0, "/home/chanwoo/Sources/gwansang/GenderRecognition/")
#sys.path.insert(0, "/home/chanwoo/Sources/gwansang/AgeEstimation/")
#sys.path.insert(0, "/home/chanwoo/Sources/gwansang/EmotionEstimation/")

import VGG_RGB
import AgeEstimation_prediction
import emotion_cnn

Packages loaded
['trainlabel', 'trainimg', 'imgsize', 'testimg', 'testlabel']
2614 train images loaded
1743 test images loaded
2304 dimensional input
Image size is [48 48]
8 classes
NETWORK READY


In [2]:
batchSize = 1
GenderImageSize = 224
AgeImageSize = 48
EmotionImageSize = 48

epochNum = 0
num_class = 2
learning_rate = 1e-6

In [3]:
#font = cv2.FONT_HERSHEY_SIMPLEX

directoryName = "face data/large"

# check points
GenderCheckPoint_dir = "GenderRecognition/"
AgeCheckPoint_dir = "AgeEstimation/"
EmotionCheckPoint_dir = "EmotionEstimation/"


### Gender Recognition Model

In [4]:
# Create Image space
GenderValidateImages = np.zeros([1, GenderImageSize, GenderImageSize, 3], dtype=np.float32)

# Create Gender Class
vgg_rgb = VGG_RGB.VGG_RGB()


# GenderRecognision input arguments
GenderImage = tf.placeholder(tf.float32, [batchSize, GenderImageSize, GenderImageSize, 3])
GenderLabel = tf.placeholder(tf.int32, [batchSize, 1])
Gender_is_train = tf.placeholder(tf.bool)


# building each model 
GenderRecognitionSession = tf.Session()
vgg_rgb.build(GenderImage, GenderLabel, Gender_is_train, scope="VGG_RGB",  num_classes=num_class, debug=True)

# load model
GenderRecognitionSaver = tf.train.Saver()
Genderckpt = tf.train.get_checkpoint_state(GenderCheckPoint_dir)

if Genderckpt and Genderckpt.model_checkpoint_path:
    GenderRecognitionSaver.restore(GenderRecognitionSession, Genderckpt.model_checkpoint_path)
    print ('load Gender model') 
    
else:
    print ('load fail')


npy file loaded
construction FCN32VGG_RGB
Layer name: conv1_1
Layer shape: (3, 3, 3, 64)
Layer name: conv1_2
Layer shape: (3, 3, 64, 64)
Layer name: conv2_1
Layer shape: (3, 3, 64, 128)
Layer name: conv2_2
Layer shape: (3, 3, 128, 128)
Layer name: conv3_1
Layer shape: (3, 3, 128, 256)
Layer name: conv3_2
Layer shape: (3, 3, 256, 256)
Layer name: conv3_3
Layer shape: (3, 3, 256, 256)
Layer name: conv4_1
Layer shape: (3, 3, 256, 512)
Layer name: conv4_2
Layer shape: (3, 3, 512, 512)
Layer name: conv4_3
Layer shape: (3, 3, 512, 512)
Layer name: conv5_1
Layer shape: (3, 3, 512, 512)
Layer name: conv5_2
Layer shape: (3, 3, 512, 512)
Layer name: conv5_3
Layer shape: (3, 3, 512, 512)
Layer name: fc6
Layer shape: [25088, 4096]
Layer name: fc7
Layer name: fc8
Layer shape: [4096, 1000]
shape of logits: (1, 2)
shape of labels: (1,)
load Gender model


### Emotion Estimation Model

In [5]:
# Create image space
EmotionValidateImages = np.zeros([1, EmotionImageSize, EmotionImageSize], dtype=np.float32)

# Create Emotion Class 
#EmotionEstimation = Emotion_Estimation()

#EmotionEstimation input arguments
EmotionInputImage = tf.placeholder( tf.float32, [batchSize, EmotionImageSize, EmotionImageSize ] )
keepratio = tf.placeholder(tf.float32)


# building model 
EmotionEstimationSession = tf.Session()
EmotionPred = emotion_cnn.conv_basic( EmotionInputImage )

# load model
EmotionEstimationSaver = tf.train.Saver(emotion_cnn.weight_list)

Emotionckpt = tf.train.get_checkpoint_state(EmotionCheckPoint_dir)

if Emotionckpt and Emotionckpt.model_checkpoint_path:
    EmotionEstimationSaver.restore(EmotionEstimationSession, Emotionckpt.model_checkpoint_path)
    print ('load emotion model' )

load emotion model


### Age Estimation Model

In [4]:
# create image space 
AgeValidateImages = np.zeros([1, AgeImageSize, AgeImageSize ], dtype=np.float32)

# Create Age Class
#AgeEstimation = AgeEstimation_prediction.AgeEstimation()

# AgeEstimation input arguments
#X=tf.placeholder(dtype=tf.float32, shape=[None,48,48,1])
AgeInputImage = tf.placeholder(tf.float32, [batchSize, AgeImageSize, AgeImageSize, 1] )


# building model
AgeEstimationSession = tf.Session()
AgePred = AgeEstimation_prediction.model( AgeInputImage )

# load model
print AgeEstimation_prediction.weight_list
AgeEstimationSaver = tf.train.Saver(AgeEstimation_prediction.weight_list)


Ageckpt = tf.train.get_checkpoint_state(AgeCheckPoint_dir)

if Ageckpt and Ageckpt.model_checkpoint_path:
    AgeEstimationSaver.restore(AgeEstimationSession, Ageckpt.model_checkpoint_path)
    print ('load Age model' )



[<tensorflow.python.ops.variables.Variable object at 0x7f8b081daa90>, <tensorflow.python.ops.variables.Variable object at 0x7f8b081de2d0>, <tensorflow.python.ops.variables.Variable object at 0x7f8b0817aed0>, <tensorflow.python.ops.variables.Variable object at 0x7f8b0819d6d0>, <tensorflow.python.ops.variables.Variable object at 0x7f8b0818d9d0>, <tensorflow.python.ops.variables.Variable object at 0x7f8b081dae90>, <tensorflow.python.ops.variables.Variable object at 0x7f8af006bc50>, <tensorflow.python.ops.variables.Variable object at 0x7f8af005bfd0>, <tensorflow.python.ops.variables.Variable object at 0x7f8af005b290>, <tensorflow.python.ops.variables.Variable object at 0x7f8af005bf90>, <tensorflow.python.ops.variables.Variable object at 0x7f8af000b250>, <tensorflow.python.ops.variables.Variable object at 0x7f8ae82ff6d0>, <tensorflow.python.ops.variables.Variable object at 0x7f8af003c490>]
load Age model


In [7]:
    
print "camera start =========================="

import numpy as np
import cv2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
newXSize = 100
newYSize = 100

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    frame = cv2.flip(frame,1)
    height, width, channels = frame.shape
    recframe = frame.copy()
    roi_color = np.zeros((1, 1, 3), np.uint8)

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    key = cv2.waitKey(1)

    for (x, y, w, h) in faces:
    if(w<width/5):
            break
        cv2.rectangle(recframe, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        crop_img = frame[y:y + h, x:x + w]

        # Capture
        if key == ord('c'):
            cv2.imshow("roi", crop_img)
            resize_Image = cv2.resize(crop_img,(newXSize, newYSize))

            ##############################
            ##############################
            ##############################
            # add deep learning function #
            #    using "resize_Image"    #
            ##############################
            ##############################
            ##############################
            
            # resizing images
            Gender_resized_img = cv2.resize(crop_img, (GenderImageSize, GenderImageSize, 3)) 
            Age_resized_img = cv2.resize(crop_img, (AgeImageSize, AgeImageSize ))
            Emotion_resized_img = cv2.resize(crop_img, (EmotionImagesize, EmotionImageSize))

            #rgb2gray emotion and age images
            #.....
            #

            GenderValidateImages[0, :, :, :] = Gender_resized_img
            AgeValidateImages[0, :, :] = Age_resized_img
            EmotionValidateImages[0, :, :] = Emotion_resized_img

                        # Predict gender
            result = GenderRecognitionSession.run(vgg_rgb.pred, feed_dict={X: ValidateImages, Gender_is_train: False})
            AgeResult = AgeEstimationSession.run( AgePred, feed_dict={AgeInputImage: AgeValidateImages } )
            EmotionResult = EmotionEstimationSession.run( EmotionPred, feed_dict={EmotionInputImage: EmotionValidateImages } )
            
            print "gen : ", result
            print "age : ", AgeResult
            print "emo : ", EmotionResult
            
            predict = result[0]

            # print strings
            print ("predict : " ) + str(predict)
            if predict==0:
                text = "Male"
            else:
                text = "Female"

            # Draw rectangle and put text on frame
            cv2.rectangle(frame, (x,y), (x2,y2), (0,255,0),3)
            cv2.putText(frame,text,(x2+10,y2-10), font, 1,(0,0,0),3)
            break

    # Display the resulting frame
    cv2.imshow('frame',recframe)

    # EXIT
    if key == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

GenderRecognitionSession.close()
AgeEstimationSession.close()
EmotionEstimationSession.close()



IndentationError: expected an indented block (<ipython-input-7-0a0b4cd960f6>, line 15)